<a href="https://colab.research.google.com/github/harish143coding/DaSciUd/blob/develop/Pyspark_learnings_nb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Executor cell


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
spark = SparkSession.builder.appName("learningsession").getOrCreate()

In [3]:
#start with D12
data = [("a", "el", 11),
        ("b", "el", 12),
        ("c", "fur", 13),
        ("d", "fur", 14),
        ("e", "el", 15),
        ("f", "fur", 16)
]
df  = spark.createDataFrame(data, ["customer", "category", "amount"])
df.show()

+--------+--------+------+
|customer|category|amount|
+--------+--------+------+
|       a|      el|    11|
|       b|      el|    12|
|       c|     fur|    13|
|       d|     fur|    14|
|       e|      el|    15|
|       f|     fur|    16|
+--------+--------+------+

